# Model Class

In [2]:
# Date: 3/17/2022
import numpy as np

In [7]:
class Model:
    def __init__(self, totalAllocation, numberOfInvestors, numberOfCategories, skillDistribution, effortFunction, categoryPreferences, utilityFunction):
        self.totalAllocation = totalAllocation
        self.numberOfInvestors = numberOfInvestors
        self.numberOfCategories = numberOfCategories
        self.distributionToNp = {
            'uniform': lambda *args: np.random.random((args)) * 10,
            'gaussian': lambda *args: np.random.normal(5, 1, (args)),
            'exponential': lambda *args: np.random.exponential(5, (args)),
        }
        self.distributions = list(self.distributionToNp.keys())
        self.distributionsStr = ", ".join(self.distributions[:-1]) + ', or ' + self.distributions[-1]
        self.effortFunctionsToNp = {
            'constant': lambda x: np.ones(x.shape),
            'log': lambda x: np.log(x),
            'linear': lambda x: x,
            'exp': lambda x: np.exp(x),
            'exponential': lambda x: np.exp(x),
        }
        self.effortFunctions = list(self.effortFunctionsToNp.keys())
        self.effortFunctionsStr = ", ".join(self.effortFunctions[:-1]) + ', or ' + self.effortFunctions[-1]
        self.utilityFunctionsToNp = {
            'identity': lambda x: np.ones(x.shape),
            'log': lambda x: np.log(x),
        }
        self.utilityFunctions = list(self.utilityFunctionsToNp.keys())
        self.utilityFunctionsStr = ", ".join(self.utilityFunctions[:-1]) + ', or ' + self.utilityFunctions[-1]
        
        self.currSkillDistribution = skillDistribution.lower()
        assert self.currSkillDistribution in self.distributions, f"Distribution Error: Skill distribution '{skillDistribution}' not recognized, try {self.distributionsStr} instead."
        self.currEffortFunction = effortFunction.lower()
        assert self.currEffortFunction in self.effortFunctions, f"Function Error: Effort function '{effortFunction}' not recognized, try {self.effortFunctionsStr} instead."
        
        self.currCategoryPreferences = categoryPreferences
        assert self.currCategoryPreferences.shape == (self.numberOfCategories,), f"Category Preferences Error: Category preferences '{categoryPreferences}' has dimension {categoryPreferences.shape} which does not match the number of categories ({self.numberOfCategories})."
        self.currUtilityFunction = utilityFunction.lower()
        assert self.currUtilityFunction in self.utilityFunctions, f"Function Error: Utility function '{utilityFunction}' not recognized, try {self.utilityFunctionsStr} instead."
        
        self.currAllocation = None
        self.currSkills = None
    
    @property
    def currHelp(self):
        if self.currAllocation is not None:
            effortFunctionFunc = self.effortFunctionsToNp[self.currEffortFunction]
            effortMatrix = effortFunctionFunc(self.currAllocation.reshape(self.currAllocation.shape[0],1))
            return effortMatrix * self.currSkills
        return None
    
    @property
    def currTotalHelp(self):
        if self.currHelp is not None:
            return self.currHelp.sum(axis=0)
        return None
    
    @property
    def currTotalUtility(self):
        if self.currTotalHelp is not None:
            utilityFunctionFunc = self.utilityFunctionsToNp[self.currUtilityFunction]
            return self.currCategoryPreferences @ utilityFunctionFunc(self.currTotalHelp)
        return None
    
    def __call__(self):
        self.currAllocation = self.makeAllocation()
        self.currSkills = self.makeSkills()
        return self.currAllocation, self.currSkills, self.currHelp, self.currTotalHelp, self.currTotalUtility
    
    def __repr__(self):
        return f"Model({self.totalAllocation}, {self.numberOfInvestors}, {self.numberOfCategories}, {self.currSkillDistribution}, {self.currEffortFunction}, {self.currCategoryPreferences}, {self.currUtilityFunction})"
    
    def __str__(self):
        return f"Model({self.totalAllocation=}, {self.numberOfInvestors=}, {self.numberOfCategories=}, {self.currSkillDistribution=}, {self.currEffortFunction=}, {self.currCategoryPreferences=}, {self.currUtilityFunction=}, {self.currHelp=}, {self.currTotalHelp=}, {self.currTotalUtility=})"
    
    def makeAllocation(self):
        arr = np.random.random(self.numberOfInvestors)
        arr /= arr.sum()
        return arr * self.totalAllocation
    
    def makeSkills(self):
        skillDistributionFunc = self.distributionToNp[self.currSkillDistribution]
        skillMatrix = skillDistributionFunc(self.numberOfInvestors, self.numberOfCategories)
        return skillMatrix

In [8]:
# Allocation, Skills, Help, TotalHelp, Total Utility
m1 = Model(
    totalAllocation = 1000,
    numberOfInvestors = 10,
    numberOfCategories = 5,
    skillDistribution = 'Exponential',
    effortFunction = 'log',
    categoryPreferences = np.arange(1,6)[::-1],
    utilityFunction = 'log'
)

In [11]:
m1

Model(1000, 10, 5, exponential, log, [5 4 3 2 1], log)

In [12]:
ca, cs, ch, th, tu = m1()

In [9]:
tu

81.99767707445486

# Notes

$A$ = "Total Allocation" ($1000$ shares)

We have vector $a$, where $a_i$ sum to $A$ -- dimension of $a$ is number of investors ($N$)

$N$ = "Total Investors" ($10$)

For an investor $v_i$ (1 <= i <= N), we have the following...

there are $k$ categories of skill, and investor $v_i$ has a single number conveying his skill in each category

Skill $s_{i,k}$ is generated from a chosen distribution (say, gaussian or exponential or uniform)

Help $e(s_{i,k}, a)$ is a function of $a$ for each investor/category pair (explictly, $e(s_{i,k}, a) = s_{i,k} * a$ in the linear case; the log case is $e(s_{i,k}, a) = s_{i,k} * log(a)$; the exponential case is $e(s_{i,k}, a) = s_{i,k} * \exp(a)$; constant case is $e(s_{i,k}, a) = s_{i,k}  * c$). Help is a vector.

We will sum up 